# **Tarea 4 - CC6205 Natural Language Processing 📚**

**Integrantes: Vicente Ardiles y Rodrigo Oportot**

**Fecha límite de entrega 📆:** Martes 22 de junio.

**Tiempo estimado de dedicación: 8 hrs**

Bienvenid@s a la cuarta tarea del curso de Natural Language Processing (NLP). 
En esta tarea estaremos tratando el problema de **tagging** (generación de secuencias de etiquetas del mismo largo que la secuencia de input), el uso de **Convolutional Neural Networks** y **Recurrent Neural Networks**, e implementaremos una red usando PyTorch. 

Usen $\LaTeX$ para las fórmulas matemáticas. En la parte de programación pueden usar lo que quieran, pero la [Axiliar 3](https://www.youtube.com/watch?v=r1pt9v_vuy8&list=PLwITEwTGVF6jXRu6l55W4OedqDiLgU9OM&index=2&t=5s) les puede ser de *gran ayuda*.

**Instrucciones:**
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo Jupyter Notebook.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación.
- En el horario de auxiliar pueden realizar consultas acerca de la tarea a través del canal de Discord del curso.

Si aún no han visto las clases, se recomienda visitar los links de las referencias.

**Referencias:**

- [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/Tjgb-yQOg54), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)       
- [MEMMs and CRFs](slides/NLP-CRF.pdf) | ([tex source file](slides/NLP-CRF.tex)), [notes 1](http://www.cs.columbia.edu/~mcollins/crf.pdf), [notes 2](http://www.cs.columbia.edu/~mcollins/fb.pdf), [video 1](https://youtu.be/qlI-4lSUDkg), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/ZpUwDy6o28Y)
- [Convolutional Neural Networks](slides/NLP-CNN.pdf) | ([tex source file](slides/NLP-CNN.tex)), [video](https://youtu.be/lLZW5Fn40r8)
- [Recurrent Neural Networks](slides/NLP-RNN.pdf) | ([tex source file](slides/NLP-RNN.tex)), [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)

# Hidden Markov Models (HMM), Maximum Entropy Markov Models (MEMM) and Conditional Random Field(CRF)

### Pregunta 1 (1 pt)
Para un problema de POS tagging se define el conjunto de etiquetas $S = \{ \text{DET}, \text{NOUN}, \text{VERB}, \text{ADP} \}$ y se tiene un Hidden Markov Model con los siguientes parámetros estimados a partir de un corpus de entrenamiento:

\begin{equation}
\begin{split}
q(\text{NOUN}| \text{ VERB}, \text{DET}) &= 0.3 \\
q(\text{NOUN}|\ w, \text{DET}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
q(\text{DET}| \text{ VERB}, \text{NOUN}) &= 0.4 \\
q(\text{DET}|\ w, \text{NOUN}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
e(the|\text{ DET}) &= 0.5 \\
e(pasta|\text{ NOUN}) &= 0.6
\end{split}
\end{equation}

Luego para la oración: `the man is pouring sauce on the pasta`, se tiene una tabla de programación dinámica con los siguientes valores:
\begin{equation}
\begin{split}
\pi(7,\text{DET},\text{DET})&=0.1\\
\pi(7,\text{NOUN},\text{DET})&=0.2\\
\pi(7,\text{VERB},\text{DET})&=0.01\\
\pi(7,\text{ADP},\text{DET})&=0.5
\end{split}
\end{equation}

Con esta información, calcule el valor de $\pi(8,\text{DET},\text{NOUN})$. Puede dejar el resultado expresado como una fracción.


**Respuesta**

Por simplicidad llamaremos a los elementos de $S$ en base a sus iniciales. $(D, N, V, A)$

Por definicion tenemos que:

\begin{equation}
\pi(8,\text{DET},\text{NOUN}) = \underset{w \epsilon S_{6}}{max}\, (\pi(7,w,D) \times q(N|w,D) \times e(pasta|N))
\end{equation}

Donde $k = 8, u = DET, v = NOUN, x_{8} = pasta.$

Del enuncido tenemos que :

\begin{equation}
\begin{split}
&e(pasta|\text{NOUN}) = 0.6 \\
&q(\text{NOUN}| \text{ VERB}, \text{DET}) &= 0.3
\end{split}
\end{equation}

Las otras etiquetas $w \neq VERB$ en $q$ no maximizan el producto debido a que $q(\text{NOUN}|\ w, \text{DET})$ es 0, entonces no queda otra opcion que $w = VERB$ y por lo tanto buscamos $\pi(7,\text{VERB},\text{DET})$

Reemplazando nos queda el calculo buscado:
\begin{equation}
\begin{split}
\pi(8,\text{DET},\text{NOUN}) &= \pi(7,\text{VERB},\text{DET}) \times q(\text{NOUN}| \text{ VERB}, \text{DET}) \times e(pasta|\text{NOUN}) \\
&= 0.01 \times 0.3 \times 0.6 \\
\pi(8,\text{DET},\text{NOUN}) &= 0.0018
\end{split}
\end{equation}

### Pregunta 2 (0.5 pts)
Comente  sobre las similitudes o diferencias entre los HMMs, MEMMs y CRFs. Para esto, responda las siguientes preguntas.

#### 2.1. ¿Para qué tipo de tarea sirven? Dé dos ejemplo de este tipo de tarea y descríbalos brevemente. (0.1 pts)

**Respuesta:** 

Estos modelos sirven para asignarles etiquetas (discretas) a las palabras de una oracion,
problema que es conocido como Sequence labeling o tagging.

Dos ejemplos de este tipo de problema son:

- Part-of-speech tagging (POS) donde se etiqueta gramaticalmente cada palabra de una oracion con su clase sintaxtica.
- Named Entity Recognition (NER) que busca reconocer entidades en una oracion como lo son personas, ciudades, intituciones, marcas, etc.

#### 2.2. ¿Qué modelos usan features? ¿Qué ventajas conlleva esto? (0.1 pts)

**Respuesta:** 

MEMMs y CRFs son modelos que usan features.
La gran ventaja de las features es que uno puede codificar en el modelo la informacion sobre relaciones/eventos
entre las etiquetas y las palabras. En otras palabras codificar eventos que ocurren dentro del input.

#### 2.3. ¿Cómo maneja cada uno de los modelos las palabras con baja frecuencia en el set de train? (0.1 pts)

**Respuesta:** 

- HMMs: Las palabras con baja frecuencia se les asigna/mapea una categoria que mejor represente la palabra en base a
un conjunto finito de etiquetas previmente definido.

- MEMMs: Como el entrenamiento consiste en ajustar los valores de $\vec W$, las palabras de bajas frecuencias terminarian
siendo eventos de muy poca frecuencia, por lo que se le asignarian pesos negativos o cercanos a 0. 

- CRFs: De forma similar que en MEMMs las palabras con baja frecuencia seguirian siendo eventos de baja frecuencia y por
lo tanto a pesar de que el contexto de etiquetas sea mas grande, el peso asigndo sera bajo.

#### 2.4. ¿Qué le permite a los CRF realizar decisiones globales? ¿Qué diferencia con respecto a los MEMMs permite lograr esto? ¿Por qué los HMMs tampoco son capaces de tomar decisiones globales? (0.1 pts)

**Respuesta:** 

Que el $\vec \Phi$ es un vector de features globales, es decir, toma en cuenta la secuencia
completa de etiquetas $S$, en cambio MEMMs esta restringido a ciertas posiciones de etiquetas por lo
que los CRF tienen informacion de todo el contexto.
HMMs tampoco tiene este comportamiento global debido a que es incluso mas restrictiva que las MEMMs
debido a que las palabras solo estan condicionadas a una sola etiqueta $(P(x_{i} | S_{i}))$.

#### 2.5 Dado una secuencia de $x_1, ..., x_m$ ¿Cuántas posibles secuencias de etiquetas se pueden generar para un conjunto de etiquetas $S$ con $|S|=k$ ? ¿Analizarlas todas sería computacionalmente tratable? (0.1 pts)

**Respuesta:** 

Se pueden generar $k^{m}$ secuencias de etiquetas. Usando algoritmos de progamacion dinamica es posible que
el analisis sobre esta secuencia se haga de forma eficiente y por lo tanto computacionalmente tratable.

# Convolutional Neural Networks
### Pregunta 3 (1 pt)

Considere la frase $w_{1..7}=$ `El agua moja y el fuego quema` $=[El, agua, moja, y, el, fuego, quema]$.

La siguiente matriz de embeddings, donde la i-ésima fila corresponde al vector de embedding de la i-ésima palabra, ordenadas según aparecen en la frase. (vectores de largo 2).
\begin{equation}
E = \begin{pmatrix}
2 & 2\\
0 & -2\\
0 & 1\\
-2 & 1\\
1 & 0\\
-1 & 1\\
1 & 1
\end{pmatrix}
\end{equation}

Los siguientes 3 filtros
\begin{equation}
U = \begin{pmatrix}
-1 & 1 & 0\\
1 & 1 & 0\\
0 & 0 & -1\\
1 & -1 & -1\\
-1 & -1 & 1\\
1 & 0 & -1
\end{pmatrix}
\end{equation}

Y la función de activación
\begin{equation}
tanh = \frac{e^{2x} - 1}{e^{2x} + 1}
\end{equation}

Usando estos paramátros escriba los pasos para calcular la representación (vector) resultante de aplicar la operación de convolución (sin padding) + max pooling. ¿De qué tamaño sería la ventana que debemos usar?


**Respuesta**

Primero, consideremos que no habrá padding por lo que será una convolución del tipo narrow. Por esta razón, el vector $w_{1...7}$ se mantiene sin cambios. Usaremos ventanas de tamaño 3, por ejemplo, "El agua moja", luego "agua moja y", etc. 

- Paso 1: se concatenan las filas de la matriz de embeddings E correspondientes a cada palabra de la ventana:

\begin{equation}
El \ agua \ moja  \rightarrow (2, \ 2, \ 0, -2, \ 0, \ 1) \\
agua \ moja \ y \rightarrow (0, -2, \ 0, \ 1, -2 , \ 1) \\
moja \ y \ el \rightarrow (0, \ 1, -2, \ 1, \ 1, \ 0) \\
y \ el \ fuego \rightarrow (-2, \ 1, \ 1, \ 0, -1, \ 1) \\
el \ fuego \ quema \rightarrow (1, \ 0, -1, \ 1, \ 1, \ 1) \\
\end{equation}

Resultando en vectores de tamaño 6

- Paso 2: Los 5 vectores resultantes del paso 1 son multiplicados, cada uno, con las filas del filtro U mediante producto punto, para luego operadar aesos resultados través de la función de activación tahn(x) y obtener valores escalares:

\begin{equation}
\vec{p_{1}} = tanh((2, \ 2, \ 0, -2, \ 0, \ 1) \bullet U) =  tanh((-1, \ 6, \ 1)) \approx ( -0.762, \ 0.999, \ 0.762) \\
\vec{p_{2}} = tanh((0, -2, \ 0, \ 1, -2 , \ 1 ) \bullet U) =  tanh(( 2, -1, -4)) \approx ( 0.964, -0.762, -0.999) \\
\vec{p_{3}} = tanh((0, \ 1, -2, \ 1, \ 1, \ 0) \bullet U) = tanh(( 1, -1, \ 2)) \approx ( 0.762, -0.762, \ 0.964) \\
\vec{p_{4}} = tanh((-2, \ 1, \ 1, \ 0, -1, \ 1) \bullet U) = tanh(( 5, \ 0, -3)) \approx ( 0.999, \ 0, -0.995) \\
\vec{p_{5}} = tanh((1, \ 0, -1, \ 1, \ 1, \ 1) \bullet U) = tanh((0, -1, \ 0)) \approx ( 0, -0.762, 0) \\
\end{equation}

Paso 3: Por último, queda aplicar max pooling a la serie de vectores $\vec{p_{i}}$ para obtener el resultado final:

\begin{equation}
max(\vec{p_{1}}(1), \vec{p_{2}}(1), \vec{p_{3}}(1), \vec{p_{4}}(1), \vec{p_{5}}(1)) = max( -0.762, \ 0.964, \ 0.762, \ 0.999, \ 0) = 0.999 \\
max(\vec{p_{1}}(2), \vec{p_{2}}(2), \vec{p_{3}}(2), \vec{p_{4}}(2), \vec{p_{5}}(2)) = max( 0.999, -0.762, -0.762, \ 0 , -0.762) = 0.999 \\
max(\vec{p_{1}}(3), \vec{p_{2}}(3), \vec{p_{3}}(3), \vec{p_{4}}(3), \vec{p_{5}}(3)) = max( 0.762, -0.999, \ 0.964, -0.995, \ 0) = 0.964 \\
\end{equation}

Por lo tanto, el vector final de representación es $\vec{c} = (0.999, \ 0.999, \ 0.964)$

Posteriormente puede aplicarse una función softmax y después cross entropy como loss function.


# Recurrent Neural Networks
### Pregunta 4 (1 pt)
Usando los embeddings de dos dimensiones de la pregunta anteror, la oración `el fuego quema` la podemos representar por una secuencia de vectores $(\vec{x}_1,\vec{x}_2,\vec{x}_3)$, con $\vec{x}_i \in \mathbb{R}^{d_x}$ y $d_x=2$.

Tenemos una red recurrente *Elman* definidad como: 
\begin{equation}
\begin{split}
\vec{s}_i &= R_{SRNN}\left (\vec{x}_i, \vec{s}_{i-1}\right ) = g \left (\vec{s}_{i-1}W^s + \vec{x}_i W^x + \vec{b}\right ) \\
\vec{y}_i &= O_{SRNN}\left(\vec{s}_i\right) = \vec{s}_i \\
\end{split}
\end{equation}
donde
\begin{equation}
\vec{s}_i, \vec{y}_i \in \mathbb{R}^{d_s}, \quad W^x \in \mathbb{R}^{d_x \times d_s}, \quad W^s \in \mathbb{R}^{d_s \times d_s}, \quad \vec{b} \in \mathbb{R}^{d_s},
\end{equation}
y los vectores de estado $s_i$ son de tres dimensiones, $ds= 3$.

Sea
\begin{equation}
\begin{split}
\vec{s}_0 &= [0,0,0]\\
W^x &= \begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} \\
W^s &= \begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} \\
\vec{b} &= [0, 0, 0] \\
g(x) &= ReLu(x) = max(0, x)
\end{split}
\end{equation}

<br>

Calcule manualmente los valores de los vectores $\vec{s}_1, \vec{s}_2,\vec{s}_3$ y de $\vec{y}_1, \vec{y}_2,\vec{y}_3$.

**Respuesta**

\begin{equation}
\vec{s_{1}} = R_{SRNN}(\vec{x_{1}}, \vec{s_{0}}) = ReLu(\vec{s_{0}}W^{s}+\vec{x_{1}}W^{x}+\vec{b}) = ReLu((0,0,0)W^{s} + (1,0)W^{x} + (0,0,0)) = ReLu((0,0,0) + (0,0,1) + (0,0,0)) = ReLu((0,0,1)) = max(0,(0,0,1)) = (0,0,1) \\
\vec{s_{2}} = R_{SRNN}(\vec{x_{2}}, \vec{s_{1}}) = ReLu(\vec{s_{1}}W^{s}+\vec{x_{2}}W^{x}+\vec{b}) = ReLu((0,0,1)W^{s} + (-1,1)W^{x} + (0,0,0)) = ReLu((1,1,1) + (1,-1,-1) + (0,0,0)) = ReLu((2,0,0)) = max(0,(2,0,0)) = (2,0,0) \\
\vec{s_{3}} = R_{SRNN}(\vec{x_{3}}, \vec{s_{2}}) = ReLu(\vec{s_{2}}W^{s}+\vec{x_{3}}W^{x}+\vec{b}) = ReLu((2,0,0)W^{s} + (1,1)W^{x} + (0,0,0)) = ReLu((2,0,2) + (1,-1,1) + (0,0,0) = ReLu((3,-1,3)) = max(0,(3,-1,3)) = (3,0,3) \\
\end{equation}

\begin{equation}
\vec{y_{1}} = O_{SRNN}(\vec{s_{1}}) = \vec{s_{1}} = (0,0,1) \\
\vec{y_{2}} = O_{SRNN}(\vec{s_{2}}) = \vec{s_{2}} = (2,0,0) \\
\vec{y_{3}} = O_{SRNN}(\vec{s_{3}}) = \vec{s_{3}} = (3,0,3) \\
\end{equation}

### Pregunta 5 (0.5 pts)
¿De qué forma las RNN y las CNN logran aprender representaciones específicas
para la tarea objetivo? Compare la forma en que las RNN y las CNN aprenden con los modelos que usan *features* diseñadas manualmente.

**Respuesta**

RNN y CNN consideran la extracción de features y la clasificación como un solo trabajo, uniendo ambos procesos en una gran tarea de entrenamiento para aprender las representaciones adecuadas de la tarea objetivo. Este acercamiento que toman RNN y CNN, siendo redes neuronales como tal, es característico de Deep Learning, diferenciándose de otros modelos de clasificación donde la extracción de features es diseñada previa y manualmente al entrenamiento, entregándolas luego como entradas a dichos modelos. Esto es propio, por ejemplo, de Machine Learning supervisado. 

CNN y RNN logran aprender las representaciones específicas para la tarea objetivo de manera no supervisada en el mismo proceso de entrenamiento, a través de transformaciones no lineales, donde el resultado de cada capa de la red es entregado a la siguiente como entrada. En la capa final, la iteración acaba, entregando el resultado para ser procesado con otros métodos adicionales, como pooling, aplicaciones de funciones softmax y de loss, e incluso conectado dicho resultado a otra arquitectura que siga trabajando la información. Por ejemplo, las RNN por sí solas no son de mucha utilidad, por lo que sus outputs son parte del entrenamiento de redes más grandes que las contengan.



# Redes neuronales con PyTorch
### Pregunta 6 (2 pts)
En esta parte van a tener que implementar una red neuronal Feed Forward. Además, deberán entrenar el modelo usando uno de los datasets de TorchText. En la sección de la respuesta hay un esqueleto de lo que deben hacer, deberán completar los metodos del modelo e implementar la parte asociada al entrenamiento. Como les mencionamos en la [Auxiliar 3](https://www.youtube.com/watch?v=r1pt9v_vuy8&list=PLwITEwTGVF6jXRu6l55W4OedqDiLgU9OM&index=2&t=5s), el proceso de entrenamiento es bastante estándar, así que se pueden guiar en gran medida por los ejemplos que ahí mostramos y los que vamos a ver en las próximas auxiliares.

#### 6.1 Capa Convolucional (Opcional)
Agregue a la arquitectura una capa convolucional. Para esto puede registrar el parametro $U$ en la red y realizar el computo de la convolución en el metodo forward de la red, o puede usar la clase [`torch.nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#conv1d) de `torch`.


In [ ]:
# %%capture
# Nos aseguramos que instalemos la version anterior de torchtext 0.9.0
!pip install torch==1.8.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchtext==0.9.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 62.9MB/s eta 0:00:19tcmalloc: large alloc 1147494400 bytes == 0x560d62e4a000 @  0x7f034a092615 0x560d28a58cdc 0x560d28b3852a 0x560d28a5bafd 0x560d28b4cfed 0x560d28acf988 0x560d28aca4ae 0x560d28a5d3ea 0x560d28acf7f0 0x560d28aca4ae 0x560d28a5d3ea 0x560d28acc32a 0x560d28b4de36 0x560d28acb853 0x560d28b4de36 0x560d28acb853 0x560d28b4de36 0x560d28acb853 0x560d28b4de36 0x560d28bd03e1 0x560d28b306a9 0x560d28a9bcc4 0x560d28a5c559 0x560d28ad04f8 0x560d28a5d30a 0x560d28acb3b5 0x560d28aca7ad 0x560d28a5d3ea 0x560d28acb3b5 0x560d28a5d30a 0x560d28acb3b5
     |█████████████████               | 1055.7MB 1.3MB/s eta 0:11:57tcmalloc: large alloc 1434370048 bytes == 0x560da74a0000 @  0x7f034a092615 0x560d28a58cdc 0x560d28b3852a 0x560d28a5bafd 0x560d28b4cfed 0x560d28acf988 0x560d28aca4ae 0x560d28a5d3ea 0x560d28acf7f0 0x560d28aca4ae 0x560d28a5d3ea 0x560d28acc32a 0x560d28b4de36 0x560d28acb853 

     |████████████████████████████████| 7.1MB 6.6MB/s 
  Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [2]:
# Trabajaremos con el dataset AG_NEWS de torchtext
# https://pytorch.org/text/stable/datasets.html#ag-news
import os
import torch
from random import choice
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from random import sample

os.makedirs("data", exist_ok=True)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
train_data, test_data = AG_NEWS(root="data")

train_split = list(train_data)
test_split = list(test_data)

print("Algunos ejemplos del dataset:")
for example in sample(list(train_split), 3):
    print("\nEjemplo aleatorio:\n", example)

# Informacion relevante del dataset
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(tokenizer(x[1]) for x in train_split)
num_classes = 4

labels = list({doc[0] for doc in train_split})
label_map = {label: index for index, label in enumerate(labels)}

print("\nTamanno del vocabulario:", len(vocab))
print("Algunas palabras del vocabulario:", sample(vocab.get_itos(), 5))
print("El label map: ", label_map)
print("Los labels: ", labels)

train.csv: 29.5MB [00:00, 69.7MB/s]
test.csv: 1.86MB [00:00, 29.5MB/s]                  


Algunos ejemplos del dataset:

Ejemplo aleatorio:
 (3, 'Treasuries Up, Indirect Bidders Buy Big  NEW YORK (Reuters) - U.S. Treasury prices built on early  gains on Wednesday after an auction of five-year debt drew  surprisingly strong demand from indirect bidders, which include  foreign central banks.')

Ejemplo aleatorio:
 (3, 'Walkout goes on as GM job-cut talks begin RUESSELSHEIM - Top Opel management and labour officials began talks Monday on the plans by General Motors to axe 10,000 jobs at the carmaker, while workers at one of the company #39;s main plants in Bochum continued a walkout.')

Ejemplo aleatorio:
 (2, 'Crawford overcomes Kenteris clamour Thursday was the night every Greek had marked down in their diary weeks and months ago. It was the night Kostas Kenteris was supposed to successfully defend his 200m crown in front of his adoring public.')

Tamanno del vocabulario: 95810
Algunas palabras del vocabulario: ['financial-services', 'methodically', 'nodong', 'handbells', 'a

## Modelo con capa de Embedding

In [3]:
# De aca para abajo viene su respuesta, completen las funciones en la red
# y luego entrenen el modelo y evaluenlo usando los dataset que acaban de
# cargar
import torch
import torch.nn as nn

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_size, 
                 use_cnn=False):
        # Aca deben registrar sus parametros. A lo menos necesitan
        # una capa de embedding y un MLP basico (una capa lineal + softmax)
        super().__init__()

        # capa de embedding
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim
                                      )
        # capa de la MLP
        self.fc = nn.Linear(embed_dim, num_class)

    def forward(self, batch): # Reemplacen el *args por sus argumentos
        # Acá debe programar la pasada hacia adelante

        # La representacion de un documento sera el promedio de los
        # embeddings de sus palabras.
        # (B, N, 1) -> (B, N, E)
        h = self.embedding(batch)
        # (B, N, E) -> (B, E)
        h = h.mean(dim=1)
        # h = self.embedding(batch)
        
        # computar las capas de la red MLP
        h = self.fc(h)
        # h = F.relu(self.fc1(h))
        # h = self.fc2(h)
        
        return h

## Entrenamiento

In [4]:
# El resto de su respuesta. Acá deben programar el entrenamiento de la red

import sys
from torch.optim import SGD
from torch.utils.data import DataLoader
from itertools import zip_longest
import time

unk_token = '<unk>'
default_index = 0
if unk_token not in vocab: vocab.insert_token(unk_token, 0)
vocab.set_default_index(default_index)

# creamos lista de tensores
train_dataset, validation_dataset = [
    [
        (
            label_map[item[0]],
            torch.tensor([vocab[token] for token in tokenizer(item[1])]),
        ) for item in split
    ] for split in [train_split, test_split]
]

def generate_batch(batch):
    # hint: si definen la capa de embedding del modelo usando nn.EmbeddingBag,
    # les puede ayudar computar offsets para cada elemento del batch

    return (
        # En este caso como los labels son números, 
        # el tensor es de una sola dimension de tamanno batch_size
        torch.tensor([item[0] for item in batch]),

        # En este caso se retorna un tensor de 2 dimensiones, batch_size x N,
        # donde N es mayor largo de los ejemplo en el batch. Aca se realiza
        # padding de los ejemplos mas cortos.
        torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *[item[1] for item in batch], fillvalue=vocab["<pad>"]
                    )
                )
            )
        ),
    )

In [5]:
# Acá deben programar el entrenamiento de la red.

def train_func(train_dataset):

    # Entranamos el modelo
    train_loss = 0
    train_acc = 0
    data = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=generate_batch,
    )
    for i, (cls, text) in enumerate(data):
        optimizer.zero_grad()
        cls, text = cls.to(device), text.to(device)
        output = model(text)
        
        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        train_acc += (output.argmax(1) == cls).sum().item()

    # Ajustar el learning rate
    # scheduler.step()

    return train_loss / len(train_dataset), train_acc / len(train_dataset)

def test(test_dataset):
    test_loss = 0
    acc = 0
    data = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch
    )
    for cls, text in data:
        cls, text = cls.to(device), text.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, cls)
            test_loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return test_loss / len(test_dataset), acc / len(test_dataset)

In [6]:
print(torch.__version__)

N_EPOCHS = 10
LEARN_RATE = 2.0
STEP_SIZE = 1
BATCH_SIZE = 16
EMBED_DIM = 100
HIDDEN_SIZE = 1024

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNNClassifier(
    vocab_size=len(vocab),
    embed_dim=EMBED_DIM,
    num_class=len(labels),
    hidden_size=HIDDEN_SIZE,
    ).to(device)

print(device)

optimizer = SGD(model.parameters(), lr=LEARN_RATE)
criterion = nn.CrossEntropyLoss().to(device)

# Intenten superar un Accuracy de 90% en el conjunto de test
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_dataset)
    valid_loss, valid_acc = test(validation_dataset)

    secs = int(time.time() - start_time)
    mins = secs // 60
    secs = secs % 60

    print(
        f"Epoch: {epoch + 1}", f" | time in {mins} minutes, {secs} seconds",
    )
    print(
        f"\tLoss: {train_loss:.4f}(train)\t|"
        f"\tAcc: {train_acc * 100:.1f}%(train)"
    )
    print(
        f"\tLoss: {valid_loss:.4f}(valid)\t|"
        f"\tAcc: {valid_acc * 100:.1f}%(valid)"
    )



1.9.0+cu102
cpu
Epoch: 1  | time in 4 minutes, 0 seconds
	Loss: 0.0674(train)	|	Acc: 58.3%(train)
	Loss: 0.1253(valid)	|	Acc: 47.4%(valid)
Epoch: 2  | time in 3 minutes, 59 seconds
	Loss: 0.0482(train)	|	Acc: 73.5%(train)
	Loss: 0.0803(valid)	|	Acc: 44.9%(valid)
Epoch: 3  | time in 4 minutes, 1 seconds
	Loss: 0.0405(train)	|	Acc: 78.1%(train)
	Loss: 0.0309(valid)	|	Acc: 82.4%(valid)
Epoch: 4  | time in 4 minutes, 0 seconds
	Loss: 0.0358(train)	|	Acc: 80.6%(train)
	Loss: 0.0387(valid)	|	Acc: 78.6%(valid)
Epoch: 5  | time in 3 minutes, 58 seconds
	Loss: 0.0326(train)	|	Acc: 82.2%(train)
	Loss: 0.0263(valid)	|	Acc: 85.4%(valid)
Epoch: 6  | time in 3 minutes, 57 seconds
	Loss: 0.0312(train)	|	Acc: 83.2%(train)
	Loss: 0.0333(valid)	|	Acc: 80.8%(valid)
Epoch: 7  | time in 3 minutes, 58 seconds
	Loss: 0.0301(train)	|	Acc: 83.8%(train)
	Loss: 0.0289(valid)	|	Acc: 83.3%(valid)
Epoch: 8  | time in 3 minutes, 57 seconds
	Loss: 0.0286(train)	|	Acc: 84.6%(train)
	Loss: 0.0296(valid)	|	Acc: 82.6%(va